In [1]:
#!pip install psycopg2-binary

In [2]:
import pandas as pd
import psycopg2
import psycopg2.extras as extras

### Setup connection

In [4]:
password = None
conn = psycopg2.connect(f"host=willitfit-db.cgj5wlpfmxhc.eu-central-1.rds.amazonaws.com port=5432 user=postgres password={password} dbname=postgres")

In [ ]:
cur = conn.cursor()

### Test getting records

In [4]:
cur.execute("""
SELECT * FROM cars 
WHERE make = 'Audi'
ORDER BY make
LIMIT 3""")

In [5]:
records = cur.fetchall()
records

[('Audi',
  'A1 1.0 TFSI ULTRA SPORT 3DR HATCH 2015',
  63,
  66,
  94,
  125,
  'http://www.ridc.org.uk/sites/default/files/styles/lead_image_content_desktop/public/field/image/car/3877_5322_0_5322.jpg?itok=XSBfcKWU',
  'SLANT'),
 ('Audi',
  'A1 CITYCARVER 5DR HATCH 2020',
  66,
  77,
  98,
  134,
  'http://www.ridc.org.uk/sites/default/files/styles/lead_image_content_desktop/public/field/image/car/4459_6011_0_6011.jpg?itok=7oUHE4iO',
  'SLANT'),
 ('Audi',
  'A1 SPORTBACK 1.6 TDI SPORT 5DR HATCH 2015',
  63,
  66,
  95,
  125,
  'http://www.ridc.org.uk/sites/default/files/styles/lead_image_content_desktop/public/field/image/car/3850_5312_0_5312.jpg?itok=hXgWUDDt',
  'SLANT')]

# Upload ikea database

### CSV > Dataframe

In [18]:
df = pd.read_csv('../willitfit/data/ikea_database.csv')
df.head()

,width,height,length,weight,packages,subarticle_code,article_code,product_name
0,22,22,46,1.16,1,60484580,60484580,SOLVINDEN
1,16,16,16,0.17,1,40470329,40470329,HÖSTFEST
2,11,21,21,0.61,1,60311339,60311339,STOCKHOLM
3,29,22,56,1.81,1,50485165,50485165,VITLÖK
4,20,2,59,1.27,1,80400042,89325963,BURHULT


### Split dataframe into correct tables

In [21]:
articles_df = df[['article_code','subarticle_code','product_name','packages']]
articles_df.head(3)

,article_code,subarticle_code,product_name,packages
0,60484580,60484580,SOLVINDEN,1
1,40470329,40470329,HÖSTFEST,1
2,60311339,60311339,STOCKHOLM,1


In [23]:
subarticles_df = df[['subarticle_code','width','height','length','weight']]
subarticles_df.head(3)

,subarticle_code,width,height,length,weight
0,60484580,22,22,46,1.16
1,40470329,16,16,16,0.17
2,60311339,11,21,21,0.61


In [26]:
def execute_batch(conn, df, table, page_size=100):
    """
    Using psycopg2.extras.execute_batch() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        extras.execute_batch(cursor, query, tuples, page_size)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_batch() done")
    cursor.close()

In [28]:
#execute_batch(conn, articles_df, "articles")

execute_batch() done


In [12]:
def execute_many(conn, df, table):
    """
    Using cursor.executemany() to insert the dataframe
    """
    # Create a list of tupples from the dataframe values
    tuples = [tuple(x) for x in df.to_numpy(dtype='object')]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL query to execute
    query  = "INSERT INTO %s(%s) VALUES(%%s,%%s,%%s,%%s,%%s)" % (table, cols)
    cursor = conn.cursor()
    try:
        cursor.executemany(query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    print("execute_many() done")
    cursor.close()

In [13]:
#execute_many(conn, subarticles_df, "sub_articles")

In [46]:
cur = conn.cursor()
cur.execute("""
SELECT * FROM articles
""")
for x in cur.fetchall():
    print(x)
cur.close()

('60484580', '60484580', 'SOLVINDEN', 1)
('40470329', '40470329', 'HÖSTFEST', 1)
('60311339', '60311339', 'STOCKHOLM', 1)
('50485165', '50485165', 'VITLÖK', 1)
('89325963', '80400042', 'BURHULT', 1)
('89325963', '10417734', 'BURHULT', 2)
('30487226', '30487226', 'LINNEBÄCK', 1)
('10203610', '10203610', 'MALM', 1)
('30446373', '30446373', 'SLATTUM', 1)
('39216754', '80311060', 'FRIHETEN', 1)
('39216754', '80311734', 'FRIHETEN', 1)
('39216754', '311059  ', 'FRIHETEN', 1)
('39280684', '70428206', 'INGATORP', 1)
('39280684', '70246292', 'INGATORP', 1)
('39280684', '10362487', 'INGATORP', 1)
('79385603', '60355130', 'PAX', 4)
('79385603', '40473139', 'PAX', 4)
('79385603', '20172750', 'PAX', 3)
('79385603', '60260373', 'PAX', 3)
('79385603', '10256892', 'PAX', 1)
('79385603', '90405765', 'PAX', 1)
('79385603', '405760  ', 'PAX', 4)
('79385603', '60405762', 'PAX', 4)
('79385603', '40400765', 'PAX', 3)
('79385603', '80355068', 'PAX', 3)
('79385603', '10405566', 'PAX', 1)
('79385603', '1044653

In [50]:
cur = conn.cursor()
cur.execute("""
SELECT * FROM sub_articles
""")
for x in cur.fetchall():
    print(x)
cur.close()

(22, 22, 46, 1.16, '60484580')
(16, 16, 16, 0.17, '40470329')
(11, 21, 21, 0.61, '60311339')
(29, 22, 56, 1.81, '50485165')
(20, 2, 59, 1.27, '80400042')
(18, 3, 18, 0.07, '10417734')
(43, 17, 73, 7.7, '30487226')
(45, 12, 148, 22.55, '10203610')
(72, 10, 152, 34.0, '30446373')
(64, 27, 207, 40.55, '80311060')
(64, 27, 207, 40.55, '80311734')
(66, 13, 94, 16.01, '70428206')
(95, 13, 161, 39.3, '70246292')
(66, 13, 94, 16.01, '10362487')
(59, 6, 245, 46.84, '60355130')
(59, 6, 245, 46.84, '40473139')
(50, 2, 239, 14.19, '20172750')
(50, 2, 239, 14.19, '60260373')
(54, 3, 237, 22.57, '10256892')
(54, 3, 237, 22.57, '90405765')
(20, 3, 25, 0.41, '60405762')
(6, 4, 49, 0.28, '40400765')
(6, 4, 49, 0.28, '80355068')
(25, 5, 53, 0.9, '10405566')
(25, 5, 53, 0.9, '10446538')
(15, 5, 53, 0.62, '90263247')
(15, 5, 53, 0.62, '60246363')
(40, 5, 53, 1.22, '10277960')
(40, 5, 53, 1.22, '70257643')
(59, 4, 242, 28.07, '60346961')
(73, 40, 145, 42.35, '00311059')
(20, 3, 25, 0.41, '00405760')


### Drop nulls

In [45]:
cur = conn.cursor()
cur.execute("""
DELETE FROM articles
    WHERE packages IS NULL
""")
cur.close()

In [48]:
cur = conn.cursor()
cur.execute("""
DELETE FROM sub_articles
    WHERE subarticle_code IS NULL
""")
cur.close()

### Prepend 0's to article nums

In [52]:
cur = conn.cursor()
cur.execute("""
UPDATE sub_articles
    SET subarticle_code = concat('0', subarticle_code)
    WHERE subarticle_code LIKE '% '
    RETURNING subarticle_code
""")
for x in cur.fetchall():
    print(x)
cur.close()

In [55]:
cur = conn.cursor()
cur.execute("""
UPDATE articles
    SET subarticle_code = concat('0', subarticle_code)
    WHERE subarticle_code LIKE '% '
    RETURNING subarticle_code
""")
for x in cur.fetchall():
    print(x)
cur.close()

('00311059',)
('00405760',)
('00475786',)
('00441904',)


In [56]:
cur = conn.cursor()
cur.execute("""
UPDATE articles
    SET article_code = concat('0', article_code)
    WHERE article_code LIKE '% '
    RETURNING article_code
""")
for x in cur.fetchall():
    print(x)
cur.close()

('00475786',)
('00441904',)


### Test Join

In [57]:
cur = conn.cursor()
cur.execute("""
SELECT articles.article_code, articles.product_name, articles.packages, sub_articles.* FROM articles
JOIN sub_articles
ON (articles.subarticle_code = sub_articles.subarticle_code)
""")
for x in cur.fetchall():
    print(x)
cur.close()

('60484580', 'SOLVINDEN', 1, 22, 22, 46, 1.16, '60484580')
('40470329', 'HÖSTFEST', 1, 16, 16, 16, 0.17, '40470329')
('60311339', 'STOCKHOLM', 1, 11, 21, 21, 0.61, '60311339')
('50485165', 'VITLÖK', 1, 29, 22, 56, 1.81, '50485165')
('89325963', 'BURHULT', 1, 20, 2, 59, 1.27, '80400042')
('89325963', 'BURHULT', 2, 18, 3, 18, 0.07, '10417734')
('30487226', 'LINNEBÄCK', 1, 43, 17, 73, 7.7, '30487226')
('10203610', 'MALM', 1, 45, 12, 148, 22.55, '10203610')
('30446373', 'SLATTUM', 1, 72, 10, 152, 34.0, '30446373')
('39216754', 'FRIHETEN', 1, 64, 27, 207, 40.55, '80311060')
('39216754', 'FRIHETEN', 1, 64, 27, 207, 40.55, '80311734')
('39280684', 'INGATORP', 1, 66, 13, 94, 16.01, '70428206')
('39280684', 'INGATORP', 1, 95, 13, 161, 39.3, '70246292')
('39280684', 'INGATORP', 1, 66, 13, 94, 16.01, '10362487')
('79385603', 'PAX', 4, 59, 6, 245, 46.84, '60355130')
('79385603', 'PAX', 4, 59, 6, 245, 46.84, '40473139')
('79385603', 'PAX', 3, 50, 2, 239, 14.19, '20172750')
('79385603', 'PAX', 3, 50

In [59]:
cur = conn.cursor()
cur.execute("""
SELECT articles.article_code, articles.product_name, articles.packages, sub_articles.* FROM articles
JOIN sub_articles
ON (articles.subarticle_code = sub_articles.subarticle_code)
WHERE articles.article_code = '79385603'
""")
for x in cur.fetchall():
    print(x)
cur.close()

('79385603', 'PAX', 4, 59, 6, 245, 46.84, '60355130')
('79385603', 'PAX', 4, 59, 6, 245, 46.84, '40473139')
('79385603', 'PAX', 3, 50, 2, 239, 14.19, '20172750')
('79385603', 'PAX', 3, 50, 2, 239, 14.19, '60260373')
('79385603', 'PAX', 1, 54, 3, 237, 22.57, '10256892')
('79385603', 'PAX', 1, 54, 3, 237, 22.57, '90405765')
('79385603', 'PAX', 4, 20, 3, 25, 0.41, '60405762')
('79385603', 'PAX', 3, 6, 4, 49, 0.28, '40400765')
('79385603', 'PAX', 3, 6, 4, 49, 0.28, '80355068')
('79385603', 'PAX', 1, 25, 5, 53, 0.9, '10405566')
('79385603', 'PAX', 1, 25, 5, 53, 0.9, '10446538')
('79385603', 'PAX', 1, 15, 5, 53, 0.62, '90263247')
('79385603', 'PAX', 1, 15, 5, 53, 0.62, '60246363')
('79385603', 'PAX', 2, 40, 5, 53, 1.22, '10277960')
('79385603', 'PAX', 2, 40, 5, 53, 1.22, '70257643')
('79385603', 'PAX', 4, 20, 3, 25, 0.41, '00405760')


In [60]:
conn.commit()
cur.close()

In [61]:
cur = conn.cursor()
cur.execute("""
SELECT * FROM articles
LIMIT 5
""")
for x in cur.fetchall():
    print(x)
cur.close()

('60484580', '60484580', 'SOLVINDEN', 1)
('40470329', '40470329', 'HÖSTFEST', 1)
('60311339', '60311339', 'STOCKHOLM', 1)
('50485165', '50485165', 'VITLÖK', 1)
('89325963', '80400042', 'BURHULT', 1)
